In [2]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv(
   r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\car_dekho_Structured.csv",
    low_memory=False
)

# Drop columns with more than 50% missing values
threshold = len(df) * 0.5
df.dropna(thresh=threshold, axis=1, inplace=True)

# Helper function to convert price from various formats to float
def convert_price(price):
    try:
        price = str(price).replace('₹', '').replace(',', '').strip()
        if 'Lakh' in price:
            return float(price.replace('Lakh', '').strip()) * 100000
        return float(price)
    except (ValueError, TypeError):
        return np.nan

# Apply conversion function to the 'price' column
df['price'] = df['price'].apply(convert_price)

# Clean the 'km' column by removing unwanted characters and converting to float
df['km'] = df['km'].str.replace('Kms', '').str.replace(',', '').astype(float)

# Fill missing values for numerical columns with the median
df.fillna({
    'price': df['price'].median(),
    'ownerNo': df['ownerNo'].median(),
    'km': df['km'].median()
}, inplace=True)

# Remove the 'owner' column as it's redundant with 'ownerNo'
df.drop(columns=['owner'], inplace=True)

# Clean and extract mileage from 'top_0_value.2' column and rename it to 'mileage'
def clean_mileage(mileage):
    try:
        mileage = str(mileage).replace('kmpl', '').replace('km/kg', '').strip()
        mileage_float = float(mileage)
        return mileage_float if mileage_float < 100 else np.nan
    except (ValueError, TypeError):
        return np.nan

df['mileage'] = df['top_0_value.2'].apply(clean_mileage)

# Clean and extract seats from 'top_3_value' column and rename it to 'Seats'
def clean_seats(seats):
    try:
        seats = str(seats).replace('Seats', '').strip()
        seats_int = int(seats)
        return seats_int if seats_int < 10 else np.nan
    except (ValueError, TypeError):
        return np.nan

df['Seats'] = df['top_3_value'].apply(clean_seats)

# Drop the original columns 'top_0_value.2' and 'top_3_value'
df.drop(['top_0_value.2', 'top_3_value'], axis=1, inplace=True)

# Removing outliers from the 'price' column using IQR method
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['price'] >= (Q1 - 1.5 * IQR)) & (df['price'] <= (Q3 + 1.5 * IQR))]

# Save the cleaned dataset
output_path = r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\car_dekho_Structuredraw.csv"
df.to_csv(output_path, index=False)

print(f"Data cleaning complete. Cleaned dataset saved as '{output_path}'.")

Data cleaning complete. Cleaned dataset saved as 'C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\car_dekho_Structuredraw.csv'.


In [ ]:
import os
import pandas as pd

# Define file paths for each city's dataset
file_paths = [
    r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\kolkata_cars.xlsx",
    r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\delhi_cars.xlsx",
    r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\bangalore_cars.xlsx",
     r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\chennai_cars.xlsx",
     r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\hyderabad_cars.xlsx",
     r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3\jaipur_cars.xlsx"
]

# Function to load and concatenate datasets
def load_and_concatenate_datasets(file_paths):
    dataframes = [pd.read_excel(file_path) for file_path in file_paths]
    combined_df = pd.concat(dataframes, ignore_index=True)
    return combined_df

# Load and concatenate the data
combined_df = load_and_concatenate_datasets(file_paths)

# Define the output directory and file name for the combined dataset
output_dir = r"C:\Users\NAGARAJAN K\Desktop\car dhekomethod3"
output_file = 'car_dekho_Structured2.csv'
output_path = os.path.join(output_dir, output_file)

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the combined DataFrame to a CSV file
combined_df.to_csv(output_path, index=False)

print(f"All datasets concatenated and saved to {output_path}")